In [1]:
import psycopg2
conn = psycopg2.connect(
    dbname="madmajors",
    user="namans",
    password="aRAY_YAR12!",
    host="db",
    port="5432"
)
cursor = conn.cursor()

In [2]:
course_codes = [
    "ASIALANG103",
    "COMP SCI200",
    "MATH 234",
    "PHYSICS 201",
    "ASIALANG104",
    "COMP SCI300",
    "MATH 240",
    "MATH 340",
    "PHYSICS 202",
    "COMP SCI252",
    "COMP SCI320",
    "COMP SCI400",
    "ASIALANG203",
    "COMP SCI354",
    "COMP SCI577",
    "GENETICS133",
    "STAT 240",
    "ASIALANG204",
    "COMP SCI537",
    "COMP SCI540",
    "MUSIC 151",
    "STAT 340",
    "ANTHRO 104",
    "ASIAN 358",
    "COMP SCI544",
    "COMP SCI564"
]


In [5]:
compl_course_tuples = [(code.replace(" ", "")[:-3], code.replace(" ", "")[-3:]) for code in course_codes]

print(compl_course_tuples)

[('ASIALANG', '103'), ('COMPSCI', '200'), ('MATH', '234'), ('PHYSICS', '201'), ('ASIALANG', '104'), ('COMPSCI', '300'), ('MATH', '240'), ('MATH', '340'), ('PHYSICS', '202'), ('COMPSCI', '252'), ('COMPSCI', '320'), ('COMPSCI', '400'), ('ASIALANG', '203'), ('COMPSCI', '354'), ('COMPSCI', '577'), ('GENETICS', '133'), ('STAT', '240'), ('ASIALANG', '204'), ('COMPSCI', '537'), ('COMPSCI', '540'), ('MUSIC', '151'), ('STAT', '340'), ('ANTHRO', '104'), ('ASIAN', '358'), ('COMPSCI', '544'), ('COMPSCI', '564')]


In [12]:
import re

# Input string
input_str = """
SELECT FROM: COMP SCI 407,412,425,435,471,475,506,513,
COMP SCI 514,518,520,524,525,526,532,533,534,536,
COMP SCI 537,538,539,540,541,542,544,552,558,559,561
OR E C E 561 COMP SCI 565,566(SP25 OR AFTER),567,
COMP SCI 570,571,576,577,579,620,635,639,620,640,642
"""

# Initialize a list to hold course tuples
course_tuples = []

# Find all occurrences of a course prefix followed by numbers
matches = re.findall(r"([A-Z ]+?)(\d{3}(?:,\d{3})*)", input_str)

for name, numbers in matches:
    # Clean up the course name
    name = name.strip().replace(" ", "")
    
    # Split numbers by commas and create tuples
    course_numbers = numbers.split(',')
    for number in course_numbers:
        course_tuples.append((name, number))

dict_course_tuples = {}
dict_course_tuples[1] = course_tuples
print(dict_course_tuples)

{1: [('COMPSCI', '407'), ('COMPSCI', '412'), ('COMPSCI', '425'), ('COMPSCI', '435'), ('COMPSCI', '471'), ('COMPSCI', '475'), ('COMPSCI', '506'), ('COMPSCI', '513'), ('COMPSCI', '514'), ('COMPSCI', '518'), ('COMPSCI', '520'), ('COMPSCI', '524'), ('COMPSCI', '525'), ('COMPSCI', '526'), ('COMPSCI', '532'), ('COMPSCI', '533'), ('COMPSCI', '534'), ('COMPSCI', '536'), ('COMPSCI', '537'), ('COMPSCI', '538'), ('COMPSCI', '539'), ('COMPSCI', '540'), ('COMPSCI', '541'), ('COMPSCI', '542'), ('COMPSCI', '544'), ('COMPSCI', '552'), ('COMPSCI', '558'), ('COMPSCI', '559'), ('COMPSCI', '561'), ('ORECE', '561'), ('COMPSCI', '565'), ('COMPSCI', '566'), ('COMPSCI', '570'), ('COMPSCI', '571'), ('COMPSCI', '576'), ('COMPSCI', '577'), ('COMPSCI', '579'), ('COMPSCI', '620'), ('COMPSCI', '635'), ('COMPSCI', '639'), ('COMPSCI', '620'), ('COMPSCI', '640'), ('COMPSCI', '642')]}


In [14]:
credits_left = 18
semesters_left = 3
max_creds_sem = 18
max_creds_take = semesters_left * max_creds_sem
cursor.execute("select count(*) from degree")
num_majors = cursor.fetchone()[0]
num_majors

319

In [16]:
where_clauses = " OR ".join([
    f"(c.course_name LIKE '%{dept}{number}' OR c.course_name LIKE '%{dept}/%{number}')"
    for dept, number in compl_course_tuples
])
print(where_clauses)

(c.course_name LIKE '%ASIALANG103' OR c.course_name LIKE '%ASIALANG/%103') OR (c.course_name LIKE '%COMPSCI200' OR c.course_name LIKE '%COMPSCI/%200') OR (c.course_name LIKE '%MATH234' OR c.course_name LIKE '%MATH/%234') OR (c.course_name LIKE '%PHYSICS201' OR c.course_name LIKE '%PHYSICS/%201') OR (c.course_name LIKE '%ASIALANG104' OR c.course_name LIKE '%ASIALANG/%104') OR (c.course_name LIKE '%COMPSCI300' OR c.course_name LIKE '%COMPSCI/%300') OR (c.course_name LIKE '%MATH240' OR c.course_name LIKE '%MATH/%240') OR (c.course_name LIKE '%MATH340' OR c.course_name LIKE '%MATH/%340') OR (c.course_name LIKE '%PHYSICS202' OR c.course_name LIKE '%PHYSICS/%202') OR (c.course_name LIKE '%COMPSCI252' OR c.course_name LIKE '%COMPSCI/%252') OR (c.course_name LIKE '%COMPSCI320' OR c.course_name LIKE '%COMPSCI/%320') OR (c.course_name LIKE '%COMPSCI400' OR c.course_name LIKE '%COMPSCI/%400') OR (c.course_name LIKE '%ASIALANG203' OR c.course_name LIKE '%ASIALANG/%203') OR (c.course_name LIKE '%CO

In [20]:
query = f"""
    SELECT d.degree_name,
           COUNT(*) AS course_count
    FROM degree_course dc
    JOIN course c ON dc.course_id = c.course_id
    JOIN degree d ON dc.degree_id = d.degree_id
    WHERE {where_clauses}
    GROUP BY d.degree_name
    ORDER BY course_count DESC
"""
cursor.execute(query)
rows = cursor.fetchall()


# Print results
for row in rows:
    print(f"Degree name: {row[0]}, Course Count: {row[1]}")

Degree name: Computer Sciences, BA, Course Count: 13
Degree name: Computer Sciences, BS, Course Count: 13
Degree name: Computer Sciences, Certificate, Course Count: 11
Degree name: Data Science, BA, Course Count: 10
Degree name: Data Science, BS, Course Count: 10
Degree name: Statistics, BS, Course Count: 8
Degree name: Statistics, BA, Course Count: 8
Degree name: Computer Engineering, BS, Course Count: 8
Degree name: Industrial Engineering, BS, Course Count: 8
Degree name: Engineering Physics, BS, Course Count: 8
Degree name: Biomedical Engineering, BS, Course Count: 7
Degree name: Atmospheric and Oceanic Sciences, BA, Course Count: 7
Degree name: Atmospheric and Oceanic Sciences, BS, Course Count: 7
Degree name: Molecular and Cell Biology, BS, Course Count: 7
Degree name: Molecular and Cell Biology, BA, Course Count: 7
Degree name: Materials Science and Engineering, BS, Course Count: 7
Degree name: Information Science, BS, Course Count: 7
Degree name: Information Science, BA, Course 

In [21]:
conn.commit()
cursor.close()
conn.close()